In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import smote_variants as sv
from tqdm import tqdm
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import os
from flashtext.keyword import KeywordProcessor

In [20]:
df1=pd.read_csv("dataset/street list.csv")
df2=pd.read_csv("dataset/1smoteog.csv")
# =pd.read_csv("dataset/syncosinestreet.csv")
df3=pd.read_csv("dataset/3smotecos.csv")
df4=pd.read_csv("dataset/StreetListCleaned.csv")
oglist=list(df1['street'])
smote=list(df2['0'])
cossmote=list(df3['0'])
cleanog=list(df4['Words'])
print("Number of intial words: ",len(oglist))
print("Number of intial words cleaned: ",len(cleanog))
print("Number of synthetic words generated using smote: ",len(smote))
print("Number of synthetic words generated using improvised smote ",len(cossmote))

Number of intial words:  323
Number of intial words cleaned:  261
Number of synthetic words generated using smote:  1862
Number of synthetic words generated using improvised smote  616


In [13]:
data=pd.read_csv('dataset/crawledcleaned.csv')
data

,Unnamed: 0,body,description,title,url,cleaned_website_text
0,0,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",Cocaine Drugs,http://fc3ryhftqfgwyroq7pt63f7jif4jknfrmd3pbdw...,cookie javascript trace protect privacy home q...
1,1,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",Cannabis,http://r3q6nxpmfdxhzz3oumaseyfr5qujvg4cayzvj2y...,cookie javascript trace protect privacy home q...
2,2,\n \n \n \n \n \n \n \n \n \n \n drugshop@dnmx...,"Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.","Buy Drugs - Ecstasy, LSD, Meth, Cocaine, Canna...",http://7nroowrqwdgbyajwrynxtypkdi46f22akwki73e...,home catalog question answer cart contact escr...
3,3,\n \n \n \n \n \n \n \n \n \n \n drugsshopper...,Drugs Shop - buy cannabis best marijuana hemp.,Cannabis - Seeds Cannabis - Drugs Shop hash,http://hpcphv5q3lhiebh6c765lslihne4tn6ui5dnd63...,home catalog question answer cart contact cate...
4,4,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",DRUGS,http://lnn6isw6tkbmplxe7eisjpf7nw6mmbwc7wjjuyh...,cookie javascript trace protect privacy home q...
...,...,...,...,...,...,...
20245,20245,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,"On the Internet, Forever-Mutating Debates\n",https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,january internet forever mutating debates rela...
20246,20246,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,1997 Technology Highlights,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,january technology highlights related articles...
20247,20247,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,Stock Volatility,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,credit new york times article home site index ...
20248,20248,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,Wall Street,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,volatile day volatile year wall street clean n...


In [14]:
data.isnull().sum()

Unnamed: 0                 0
body                       0
description             2539
title                      0
url                        0
cleaned_website_text      16
dtype: int64

In [15]:
data.dropna(subset=['cleaned_website_text'],inplace=True)
data.isnull().sum()

Unnamed: 0                 0
body                       0
description             2530
title                      0
url                        0
cleaned_website_text       0
dtype: int64

In [16]:
from flashtext.keyword import KeywordProcessor
kp0=KeywordProcessor()
c=0
for word in oglist:
    kp0.add_keyword(word)
    c=c+1
print(c)
def percentage1(dum0,dumx):
    try:
        ans=float(dumx)/float(dum0)
        ans=ans*100
    except:
        return 0
    else:
        return ans
def find_category(x):
    try:
        Total_matches=kp0.extract_keywords(x)
        y0=len(Total_matches)
        n=len(x)
        per=percentage1(n,y0)
        if y0==0:
            matched_catkeywords='None'
        matched_catkeywords=Total_matches
        return y0,matched_catkeywords,per
    except:
        print(x)
Total=[]
words=[]
per=[]
df=data
for i in df['cleaned_website_text']:
    Total.append(find_category(i)[0])
    words.append(find_category(i)[1])
    per.append(find_category(i)[2])
df['no of matched keywords']=Total
df['words matched']=words
df['percent']=per
cat=[]
for i in per:
    if i>0:
        cat.append("drug")
    else:
        cat.append("non drug")
    
df['category']=cat
df['category'].value_counts()

323


drug        11745
non drug     8489
Name: category, dtype: int64

In [17]:
df.to_csv("datawithoglist.csv")

In [18]:
kp0=KeywordProcessor()
c=0
for word in smote:
    kp0.add_keyword(word)
    c=c+1
print(c)
def percentage1(dum0,dumx):
    try:
        ans=float(dumx)/float(dum0)
        ans=ans*100
    except:
        return 0
    else:
        return ans
def find_category(x):
    try:
        Total_matches=kp0.extract_keywords(x)
        y0=len(Total_matches)
        n=len(x)
        per=percentage1(n,y0)
        if y0==0:
            matched_catkeywords='None'
        matched_catkeywords=Total_matches
        return y0,matched_catkeywords,per
    except:
        print(x)
Total=[]
words=[]
per=[]
df1=data
for i in tqdm(df1['cleaned_website_text']):
    Total.append(find_category(i)[0])
    words.append(find_category(i)[1])
    per.append(find_category(i)[2])
df1['no of matched keywords']=Total
df1['words matched']=words
df1['percent']=per
cat=[]
for i in per:
    if i>0:
        cat.append("drug")
    else:
        cat.append("non drug")
    
df1['category']=cat
df1['category'].value_counts()

1862


100%|████████████████████████████████████████████████████████████████████████████| 20234/20234 [03:38<00:00, 92.47it/s]


drug        20091
non drug      143
Name: category, dtype: int64

In [ ]:
# df.to_csv("datawithsmote.csv")

In [19]:
from flashtext.keyword import KeywordProcessor
kp0=KeywordProcessor()
c=0
for word in cossmote:
    kp0.add_keyword(word)
    c=c+1
print(c)
def percentage1(dum0,dumx):
    try:
        ans=float(dumx)/float(dum0)
        ans=ans*100
    except:
        return 0
    else:
        return ans
def find_category(x):
    try:
        Total_matches=kp0.extract_keywords(x)
        y0=len(Total_matches)
        n=len(x)
        per=percentage1(n,y0)
        if y0==0:
            matched_catkeywords='None'
        matched_catkeywords=Total_matches
        return y0,matched_catkeywords,per
    except:
        print(x)
Total=[]
words=[]
per=[]
df2=data
for i in tqdm(df2['cleaned_website_text']):
    Total.append(find_category(i)[0])
    words.append(find_category(i)[1])
    per.append(find_category(i)[2])
df2['no of matched keywords']=Total
df2['words matched']=words
df2['percent']=per
cat=[]
for i in per:
    if i>0:
        cat.append("drug")
    else:
        cat.append("non drug")
    
df2['category']=cat
df2['category'].value_counts()

616


100%|███████████████████████████████████████████████████████████████████████████| 20234/20234 [03:12<00:00, 105.04it/s]


drug        17565
non drug     2669
Name: category, dtype: int64

In [ ]:
# df.to_csv("datawithcossmote.csv")

In [21]:
from flashtext.keyword import KeywordProcessor
kp0=KeywordProcessor()
c=0
for word in cleanog:
    kp0.add_keyword(word)
    c=c+1
print(c)
def percentage1(dum0,dumx):
    try:
        ans=float(dumx)/float(dum0)
        ans=ans*100
    except:
        return 0
    else:
        return ans
def find_category(x):
    try:
        Total_matches=kp0.extract_keywords(x)
        y0=len(Total_matches)
        n=len(x)
        per=percentage1(n,y0)
        if y0==0:
            matched_catkeywords='None'
        matched_catkeywords=Total_matches
        return y0,matched_catkeywords,per
    except:
        print(x)
Total=[]
words=[]
per=[]
df3=data
for i in tqdm(df3['cleaned_website_text']):
    Total.append(find_category(i)[0])
    words.append(find_category(i)[1])
    per.append(find_category(i)[2])
df3['no of matched keywords']=Total
df3['words matched']=words
df3['percent']=per
cat=[]
for i in per:
    if i>0:
        cat.append("drug")
    else:
        cat.append("non drug")
    
df3['category']=cat
df3['category'].value_counts()

261


100%|███████████████████████████████████████████████████████████████████████████| 20234/20234 [03:14<00:00, 104.25it/s]


drug        18466
non drug     1768
Name: category, dtype: int64

In [22]:
df3.to_csv("datawithcleanog.csv")